# TAO ontology builder

### Environment setup

In [2]:
from owlready2 import *
world = World()
world.set_backend(filename = "./world_file.sqlite3", exclusive = False) ##use concurrently

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
import logging
logging.basicConfig( level=logging.ERROR)

In [5]:
from lib.helpers import duplicate_class_hyerarchy, base_class, base_class_merge, annotation_copier, TaoBuilder 
from lib.helpers import provenance_annotator, apply_generic_properties, generate_class_hierarchy
from lib.helpers import disjoin_class_hyerarchy, create_external_resource, labeller_by_class_name

In [7]:
import pandas as pd

In [8]:
import re

In [9]:
onto_path.append("./")

In [10]:
#skos_file = 'http://www.w3.org/2004/02/skos/core' 
skos_file = './ontologies/skos.rdf' 
skos_ontology = world.get_ontology(skos_file).load()
#skos = skos_ontology.get_namespace("http://www.w3.org/2004/02/skos/core#")
skos = skos_ontology.get_namespace("http://www.w3.org/2004/02/skos/core#")

## DBpedia namespaces
dbr = world.get_ontology("http://dbpedia.org/resource/")
dbo = world.get_namespace("http://dbpedia.org/ontology/")

## Wikipedia namespace
wiki = world.get_namespace("http://en.wikipedia.org/wiki/")

## Geonames namespace
gn_ontology = world.get_ontology("./ontologies/geonames_ontology_v3.2.rdf").load()
gn = gn_ontology.get_namespace("https://www.geonames.org/ontology")

## GoodRelations namespace
gr_ontology = world.get_ontology("./ontologies/GoodRelations.rdf").load()
#gr_ontology = world.get_ontology("http://purl.org/goodrelations/v1#").load()
gr = gr_ontology.get_namespace("http://purl.org/goodrelations/v1#")

acco_ontology = world.get_ontology("./ontologies/acco.rdf").load()
acco = acco_ontology.get_namespace("http://purl.org/acco/ns#")

hontology = world.get_ontology("./ontologies/hontology.rdf").load()
ho = hontology.get_namespace("http://ontolp.inf.pucrs.br/Recursos/downloads-Hontology.php#")
#ho = hontology.get_namespace("http://www.semanticweb.org/ontologies/2010/3/hotel-15072010.owl#")


schema_ontology = world.get_ontology("./ontologies/schema.rdf").load()
#schema_ontology = world.get_ontology("./ontologies/schemaorg.owl").load()
#schema_ontology = world.get_ontology("./ontologies/schemaorg_http.owl").load()
schema = schema_ontology.get_namespace("http://schema.org/")

dcterms = world.get_ontology("http://purl.org/dc/terms/").load()
dct = dcterms.get_namespace("http://purl.org/dc/terms/")

#geonames_mappings = world.get_ontology("https://www.geonames.org/ontology/mappings_v3.01.rdf").load()

tao_ontology = world.get_ontology("./ontologies/tao_base.rdf").load()
#tao_ontology = world.get_ontology("http://purl.org/tao_solo/ns#")
tao_solo = tao_ontology.get_namespace("http://purl.org/tao/ns#")
#schema_tao = tao_ontology.get_namespace("http://schema.org/")

* Owlready2 * WARNING: DataProperty https://www.geonames.org/ontology#alternateName belongs to more than one entity types: [owl.AnnotationProperty, owl.DatatypeProperty, skos.altLabel]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty https://www.geonames.org/ontology#officialName belongs to more than one entity types: [owl.AnnotationProperty, owl.DatatypeProperty, skos.prefLabel]; I'm trying to fix it...
* Owlready2 * WARNING: AnnotationProperty http://schema.org/disambiguatingDescription belongs to more than one entity types: [owl.AnnotationProperty, schema.org.description]; I'm trying to fix it...
* Owlready2 * WARNING: AnnotationProperty http://schema.org/layoutImage belongs to more than one entity types: [owl.AnnotationProperty, schema.org.image]; I'm trying to fix it...
* Owlready2 * WARNING: AnnotationProperty http://schema.org/logo belongs to more than one entity types: [owl.AnnotationProperty, schema.org.image]; I'm trying to fix it...
* Owlready2 * WARNING: Annotat

In [11]:
acco.value.python_name = "acco_value"
gr.__getitem__('name').python_name = "gr_name"
#skos.__getitem__('altLabel').python_name = "skos_altLabel"
#skos.__getitem__('exactMatch').python_name = "skos_exactMatch"


### Note; skos:altLabel, skos:prefLabel, skos:related are declared in the tao base rdf file
### I couldn't use them with the python_name technique

In [12]:
tao_builder = TaoBuilder(world)

### SeeAlso wrapper
We define a new class wrapper that add a rdfs.seeAlso property to connect the imported class to the original one.

In [13]:
def seeAlso_wrapper(owl_class, ancestor, namespace, copy = True):
    text_comment = "" 
    return apply_generic_properties(owl_class, ancestor, namespace, copy, rdfs.seeAlso, text_comment) ## disabilitiamo i commenti alle proprietà

def seeAlso_merger_wrapper(owl_class, ancestor, namespace, copy = True):
    text_comment = "" 
    return apply_generic_properties(owl_class, ancestor, namespace, copy, rdfs.seeAlso, text_comment, class_builder=base_class_merge) ## disabilitiamo i commenti alle proprietà


# Tourist Locations

### Create tao:TouristLocation class
We create a TouristLocation class as the root of a class hierarchy that models places interesting from a tourist point of view.

In [14]:
with tao_solo:
    class TouristLocation(schema.Place, gn.Feature):
        label = [locstr("Tourist location", lang = "en")]
        comment = """A location is a point or area of interest from a tourist point of view, which a particular product or service is available, e.g. a museum, a beach, a bus stop, a gas station, or a ticket booth. The difference to gr:BusinessEntity is that the gr:BusinessEntity is the legal entity (e.g. a person or corporation) making the offer, while tao:Location is the store, office, or place. A chain restaurant will e.g. have one legal entity but multiple restaurant locations. Locations are characterized by an address or geographical position and a set of opening hour specifications for various days of the week."""
        altLabel = [locstr("Point of interest", lang = "en"), locstr("Area of interest", lang = "en"), locstr("Location", lang = "en")]
        seeAlso = gr.Location
    
    class TouristDestination(gn.Feature):
        label = [locstr("Tourist destination", lang = "en")]
        comment = """A tourist destination. A TouristDestination is defined as a Place that contains, or is colocated with, one or more TouristLocation and LodgingFacility, often linked by a similar theme or interest to a particular tourist audience. The [UNWTO](http://www2.unwto.org/) defines Destination (main destination of a tourism trip) as the place visited that is central to the decision to take the trip."""
        equivalent_to = [schema.TouristDestination]
        

### Import subclasses of schema:Place
We now import subclasses of schema:Place class as subclasses of tao:TouristLocation in TAO ontology.

In [15]:
schema_places_classes = {}
exclude_classes = [
    schema.Accommodation, acco.Accommodation, tao_solo.LodgingFacility, schema.LodgingBusiness,
    schema.AnimalShelter, schema.ArchiveOrganization,
    schema.AutoBodyShop,schema.AutoDealer,schema.AutoPartsStore, schema.AutomotiveBusiness, schema.MotorcycleDealer, schema.Campground,
    schema.ChildCare, schema.EmploymentAgency, schema.FinancialService, schema.HomeAndConstructionBusiness, schema.LegalService, 
    schema.MedicalBusiness, schema.ProfessionalService, schema.RadioStation, schema.RealEstateAgent, schema.RecyclingCenter, schema.SelfStorage, 
    schema.AutoPartsStore, schema.BikeStore, schema.FurnitureStore, schema.GardenStore, schema.HardwareStore, schema.HomeGoodsStore, 
    schema.MovieRentalStore, schema.OfficeEquipmentStore, schema.PawnShop, schema.PetStore, schema.TireShop, schema.WholesaleStore,
    schema.TelevisionStation, schema.Residence,
    schema.TouristDestination, ## We define our Tourist destination class
    tao_solo.TouristLocation ## avoid cyclic reference
] ## We do not take schema.Accommodations as TouristLocations and remove tao:Accommodation to prevent loops
duplicate_class_hyerarchy(schema.Place, schema_places_classes, tao_solo.TouristLocation,
                          class_wrapper=seeAlso_merger_wrapper, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          disjoint=False, namespace = tao_solo, copy = True, exclude = exclude_classes)

In [16]:
exclude_from_exploration = [
    tao_solo.LocalBusiness
]

disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
        tao_solo.TouristAttraction,
        tao_solo.TouristDestination,
        tao_solo.EducationalOrganization,
        tao_solo.GovernmentBuilding,
        tao_solo.EventVenue,
        tao_solo.MusicVenue
    ],
    "do_not_disjoin": [
        (tao_solo.CivicStructure,tao_solo.LandmarksOrHistoricalBuildings, tao_solo.LocalBusiness),
        #(tao_solo.TaxiStand, tao_solo.TrainStation),
        (tao_solo.TrainStation, tao_solo.BusStation, tao_solo.SubwayStation),
        (tao_solo.MovieTheater, tao_solo.Museum, tao_solo.PerformingArtsTheater, tao_solo.StadiumOrArena)
    ]
}
disjoin_class_hyerarchy(tao_solo.TouristLocation, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
#disjoin_class_hyerarchy(tao_solo.CivicStructure, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)

# Lodging Facilities

### Create TAO base classes

In [17]:
with tao_solo:
    class Hotel(tao_solo.LodgingFacility):
        ### partially defined in the base owl file for the ontology
        label = [locstr("Hotel", lang = "en")]
        altLabel = [locstr("Boutique hotel", lang = "en")]  
        comment = locstr("""A hotel is an establishment that provides lodging paid on a short-term basis. 
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Hotel)""", lang = "en")
        seeAlso = [ schema.Hotel, create_external_resource("Hotel", dbr)] ## in this way the DBPedia resource is linked
    
    class Campground(tao_solo.LodgingFacility):
        label = [locstr("Campground", lang = "en")]
        altLabel = [locstr("Campsite", lang = "en")]  
        comment = locstr("""The term camp comes from the Latin word campus, meaning "field". 
        Therefore, a campground consists typically of open pieces of ground where a camper can pitch a tent or park a camper.
        More specifically a campsite is a dedicated area set aside for camping and for which often a user fee is charged. 
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Campground)""", lang = "en")
        seeAlso = [ create_external_resource("Campsite", dbr), create_external_resource("Campground", dbr)]
    
    class Loft(tao_solo.LodgingFacility):
        label = [locstr("Loft", lang = "en")]
        comment = locstr("""A loft is a building's upper storey or elevated area in a room directly under the roof (American usage), or just an attic: a storage space under the roof usually accessed by a ladder (primarily British usage).
        A loft apartment refers to large adaptable open space, often converted for residential use (a converted loft) from some other use, often light industrial.
        Adding to the confusion, some converted lofts include upper open loft areas. 
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Loft)""", lang = "en")
        seeAlso = [ create_external_resource("Loft", dbr)]
       
    class Apartment(tao_solo.LodgingFacility):
        label = [locstr("Apartment", lang = "en")]
        altLabel = [locstr("Condo", lang = "en"), locstr("Apartments", lang = "en"), locstr("Flat", lang = "en"), locstr("Condominium", lang = "en")]  
        comment = locstr("""An apartment (American English), or flat (British English, Indian English, South African English), is a self-contained housing unit (a type of residential real estate) that occupies part of a building, generally on a single story.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Apartment) """, lang = "en")
        seeAlso = [create_external_resource("Apartment", dbr), schema.Apartment]
    
    class Aparthotel(tao_solo.Hotel, tao_solo.Apartment):
        label = [locstr("Aparthotel", lang = "en")]
        altLabel = [locstr("Condo hotel", lang = "en"), 
                    locstr("Serviced apartments", lang = "en"),
                    locstr("Serviced apartment", lang = "en"),
                    locstr("Apartment hotel", lang = "en")]
        comment = locstr("""An apartment hotel or aparthotel (also residential hotel, or extended-stay hotel) is a serviced apartment complex that uses a hotel-style booking system. 
        It is similar to renting an apartment, but with no fixed contracts and occupants can "check out" whenever they wish.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Apartment_hotel)""", lang = "en")
        seeAlso = create_external_resource("Aparthotel", dbr)
        
    class House(tao_solo.LodgingFacility):
        label = [locstr("House", lang = "en")] 
        altLabel = [locstr("Cottage", lang = "en"), locstr("Townhouse", lang = "en"), 
                    locstr("Earth house", lang = "en"), locstr("Tiny house", lang = "en")]
        comment = locstr("""A house is a building or structure that has the ability to be occupied for habitation by humans or other creatures. 
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/House)""", lang = "en")
        seeAlso = [schema.House, create_external_resource("House",dbr)]
    
    class CountryHouseHotel(tao_solo.Hotel)  :
        label = [locstr("Country house hotel", lang = "en")]
        altLabel = [
            locstr("Country house", lang = "en"),
            locstr("Country houses", lang = "en")]
        comment = locstr("""Country House Hotel, is a hotel located in the country side, which appears more as a home than as a hotel. 
        Country house hotels thereby differentiate themselves from ordinary hotels by providing a more local and intimate experience.
        (Source: XOTELS, the free encyclopedia, see https://www.xotels.com/en/glossary/country-house-hotel)""", lang = "en")
    
    class GuestHouse(tao_solo.House):
        label = [locstr("Guest house", lang = "en")]
        altLabel = [locstr("Guesthouse", lang = "en"), locstr("Homestay", lang = "en")]      
        comment = locstr("""A guest house (also guesthouse) is a kind of lodging. In some parts of the world, guest houses are a type of inexpensive hotel-like lodging.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Guest_house)""", lang = "en")
        seeAlso = create_external_resource("Guest_house", dbr)

    class Bungalow(tao_solo.House):
        label = [locstr("Bungalow", lang = "en")]   
        comment = locstr("""A bungalow is a small house or cottage that is either single-story or has a second story built into a sloping roof (usually with dormer windows), and may be surrounded by wide verandas.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Houseboat)""", lang = "en")        
        seeAlso = create_external_resource("Bungalow", dbr)
        
    class HolidayHome(tao_solo.House):
        label = [locstr("Holiday home", lang = "en")]
        altLabel = [locstr("Vacation Home", lang = "en"),
            locstr("Vacation House", lang = "en"),
            locstr("Vacation Homes", lang = "en"),
            locstr("Holiday Homes", lang = "en")]
        comment = locstr("""A vacation rental is the renting out of a furnished apartment, house, or professionally managed resort-condominium complex on a temporary basis to tourists as an alternative to a hotel. 
        The term vacation rental is mainly used in the US. In Europe the term villa rental or villa holiday is preferred for rentals of detached houses in warm climates. 
        Other terms used are self-catering rentals, holiday homes, holiday lets (in the United Kingdom), cottage holidays (for rentals of smaller accommodation in rural locations) and gites (in rural locations in France).
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Vacation_rental and https://en.wikipedia.org/wiki/Holiday_cottage)""", lang = "en")
        seeAlso = [create_external_resource("Vacation_rental", dbr), create_external_resource("Holiday_cottage", dbr)]
    
    class Villa(tao_solo.HolidayHome):
        label = [locstr("Villa", lang = "en")]
        comment = locstr("""Originally a type of house that was originally an ancient Roman upper-class country house.
        Today indicates a particular kind of Holiday home which is often a high quality detached home in warm destinations.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Vacation_rental and https://en.wikipedia.org/wiki/Villa)""", lang = "en")
        seeAlso = [create_external_resource("Vacation_rental", dbr), create_external_resource("Villa", dbr)]
 
    class Boat(tao_solo.LodgingFacility):
        label = [locstr("Boat", lang = "en")]   
        comment = locstr("""A boat is a watercraft of a large range of types and sizes.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Boat)""", lang = "en")
        seeAlso = create_external_resource("Boat", dbr)

    class HouseBoat(tao_solo.Boat):
        label = [locstr("House boat", lang = "en")]
        altLabel = [locstr("Houseboat", lang = "en")]      
        comment = locstr("""A houseboat is a boat that has been designed or modified to be used primarily as a home.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Houseboat)""", lang = "en")
        seeAlso = create_external_resource("Houseboat", dbr)

    class Botel(tao_solo.Boat):
        label = [locstr("Boat hotel", lang = "en")]
        altLabel = [locstr("Hotelship", lang = "en"), locstr("Botel", lang = "en")]      
        comment = locstr("""A hotelship is a passenger ship which is used for a short period as a hotel. 
        A botel or boatel is a boat that serves as a hotel or hostel curently there are few permanent hotel ships.
        The word is a portmanteau of boat and hotel. (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Boat_hotel)""", lang = "en")
        seeAlso = create_external_resource("Boat_hotel", dbr)
        
    class BedAndBreakfast(tao_solo.LodgingFacility):
        label = [locstr("Bed and breakfast", lang = "en")]
        altLabel = [locstr("B&B", lang = "en"), locstr("BnB", lang = "en")]      
        comment = locstr("""A bed and breakfast (typically shortened to B&B or BnB) is a small lodging establishment that offers overnight accommodation and breakfast. 
        Bed and breakfasts are often private family homes and typically have between four and eleven rooms, with six being the average. 
        In addition, a B&B usually has the hosts living in the house. 
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Bed_and_breakfast)""", lang = "en")
        seeAlso = [schema.BedAndBreakfast, create_external_resource("Bed_and_breakfast", dbr)]
        
    class Inn(tao_solo.LodgingFacility):
        label = [locstr("Inn", lang = "en")]
        altLabel = [locstr("Locanda", lang = "it")]         
        comment = locstr("""Inns are generally establishments or buildings where travelers can seek lodging, and usually, food and drink.
        Inns are typically located in the country or along a highway.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Inn)""", lang = "en")
        seeAlso = create_external_resource("Inn", dbr)
  
    class Motel(tao_solo.LodgingFacility):
        label = [locstr("Motel", lang = "en")]
        altLabel = [locstr("Motel", lang = "it")]         
        comment = locstr("""A motel, also known as a motor hotel or motor lodge, is a hotel designed for motorists, usually having each room entered directly from the parking area for motor vehicles rather than through a central lobby.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Motel)""", lang = "en")
        seeAlso = [schema.Motel, create_external_resource("Motel", dbr)]
    
    class Pension(tao_solo.LodgingFacility):
        label = [locstr("Pension", lang = "en")]
        altLabel = [locstr("Boarding house", lang = "en")]
        comment = locstr("""A pension is a type of boarding house. A boarding house is a house (frequently a family home) in which lodgers rent one or more rooms on a nightly basis, and sometimes for extended periods of weeks, months, and years. 
        The common parts of the house are maintained, and some services, such as laundry and cleaning, may be supplied. 
        They normally provide "room and board," that is, at least some meals as well as accommodation.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Boarding_house and https://en.wikipedia.org/wiki/Pension_(lodging))""", lang = "en")
        seeAlso = [create_external_resource("Boarding_house", dbr), create_external_resource("Pension_(lodging)", dbr)]
    
    class Riad(tao_solo.LodgingFacility):
        label = [locstr("Riad", lang = "en")]
        comment = locstr("""A riad is a type of traditional Moroccan and Andalusi interior garden or courtyard associated with house and palace architecture.
        The term is nowadays often used in Morocco to refer to a hotel or guesthouse-style accommodation with shared common areas and private rooms, often within a restored traditional mansion.
        (Source: Wikipedia, the free encyclopedia, see https://en.wikipedia.org/wiki/Riad_(architecture))""", lang = "en")
        seeAlso = create_external_resource("Riad", dbr)
        
    class Resort(tao_solo.LodgingFacility):
        label = [locstr("Resort", lang = "en")]
        comment = locstr("""A resort is a place used for relaxation or recreation, attracting visitors for holidays or vacations. Resorts are places, towns or sometimes commercial establishment operated by a single company.
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Resort)""", lang = "en")
        seeAlso = create_external_resource("Resort", dbr)        

In [18]:
tao_solo.BedAndBreakfast.seeAlso

[schema.BedAndBreakfast, resource.Bed_and_breakfast]

### Import subbclasses of Hontology Accommodation as tao:LodgingFacility subclasses
We now import from Hontology some classes useful to model accommodations.

In [19]:
lodg_classes = {}
exclude_classes = [ho.Apartment, ho.BungalowAndCamping, ho.GuestHouse, ho.HolidaySettlement, ho.OtherHospitality, ho.Botel] #Classes not useful or sufficiently clear
duplicate_class_hyerarchy(ho.Accommodation, lodg_classes,  tao_solo.LodgingFacility, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          namespace = tao_solo, exclude = exclude_classes)

### Import subbclasses of schema:Accommodation as tao:LodgingFacility subclasses
We now import from Schema.org some classes useful to model accommodations.

In [20]:
lodg_classes = {}
exclude_classes = [schema.CampingPitch, schema.Room, schema.Suite, schema.SingleFamilyResidence] #Classes not useful, out of scope or not sufficiently clear
duplicate_class_hyerarchy(schema.Accommodation, lodg_classes,  tao_solo.LodgingFacility, 
                          class_wrapper = seeAlso_wrapper, namespace = tao_solo, exclude = exclude_classes)

### Import subbclasses of schema:LodgingBusiness as tao:LodgingFacility subclasses
We now import from Schema.org some classes useful to model accommodations.

In [21]:
lodg_classes = {}

exclude_classes = [schema.Campground] #Classes not useful, out of scope or not sufficiently clear
duplicate_class_hyerarchy(schema.LodgingBusiness, lodg_classes,  tao_solo.LodgingFacility, 
                          class_wrapper = seeAlso_wrapper, namespace = tao_solo, exclude = exclude_classes)

# Accommodations

In [22]:
with tao_solo:
    class Accommodation(acco.Accommodation):
        ### partially defined in the base owl file for the ontology
        label = [locstr("Accommodation", lang = "en")]
        comment = locstr("""An accommodation is a place that can accommodate human beings, e.g. a hotel room, a camping pitch, or a meeting room.
        Many accommodations are for overnight stays, but this is not a mandatory requirement.
        An accommodation is always part of a LodgingFacility (e.g. an HotelRoom is part of an Hotel, an EntireApartment is the whole Apartment (LodgingFacility) used as an accommodation.)""", lang = "en")
        seeAlso = [ schema.Accommodation ]
    
    class Room(tao_solo.Accommodation):
        label = [locstr("Room", lang = "en")]
        comment = locstr("""A room is a distinguishable space within a structure, usually separated from other spaces by interior walls.(Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Room).""", lang = "en")
        seeAlso = [ schema.Room, create_external_resource("Room", dbr)]

 
    class SharedRoom(tao_solo.Room):
        label = [locstr("Shared room", lang = "en")]
        comment = locstr("""Shared rooms are for when you don't mind sharing a space with others. 
        When you book a shared room, you'll be sleeping in a space that is shared with others and share the entire space with other people. 
        Shared rooms are popular among flexible travelers looking for new friends and budget-friendly stays.
        (Source: AirBnb, see https://www.airbnb.com/help/article/5/types-of-places-to-stay#section-heading-0-3).
        """, lang = "en")
        
    class HotelRoom(tao_solo.Room):
        ### partially defined in the base owl file for the ontology
        label = [locstr("Hotel room", lang = "en")]
        comment = locstr("""A hotel room is a single room in a hotel.""", lang = "en")
        seeAlso = [ schema.HotelRoom, acco.HotelRoom]
        
    class MeetingRoom(tao_solo.Room, acco.MeetingRoom):
        label = [locstr("Meeting room", lang = "en")]
        comment = locstr("""A meeting room, conference room, or conference hall is a room provided for singular events such as business conferences and meetings
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Conference_hall).""", lang = "en")
        seeAlso = [ schema.MeetingRoom, create_external_resource("Conference_hall", dbr)]
    
    class Suite(tao_solo.Accommodation):
        label = [locstr("Suite", lang = "en")]
        altLabel = [locstr("Guest suite", lang = "en")]
        comment = locstr("""A suite in a hotel or other public accommodation, denotes a class of luxury accommodations, the key feature of which is multiple rooms.
        (Source: Wikipedia, the free encyclopedia, see http://en.wikipedia.org/wiki/Suite_(hotel)) """, lang = "en")
        seeAlso = [ schema.Suite, acco.Suite, create_external_resource("Suite_(hotel)", dbr)]
    
    class EntirePlace(tao_solo.Accommodation):
        label = [locstr("Entire palce", lang = "en")]
        comment = locstr("""An accommodation which comprises an entire generic lodging facility.""", lang = "en")
    
    class EntireBoat(tao_solo.EntirePlace):
        label = [locstr("Entire boat", lang = "en")]
        comment = locstr("""An entire boat used as an accommodation.""", lang = "en")
    
    class EntireLoft(tao_solo.EntirePlace):
        label = [locstr("Entire loft", lang = "en")]
        comment = locstr("""An entire loft used as an accommodation.""", lang = "en")
    
    class EntireHouse(tao_solo.EntirePlace):
        pass ## defined in the base owl file for the ontology
     
    class EntireApartment(tao_solo.EntirePlace):
        pass ## defined in the base owl file for the ontology
 
    class CampingPitch(tao_solo.Accommodation):
        pass ## defined in the base owl file for the ontology

# Accommodation Amenities

In [23]:
with tao_solo:
    class LocationAmenity(Thing):
        label = locstr("Location amenity", lang = "en")
        altLabel = [
            locstr("Accommodation amenity", lang = "en"),
            locstr("Lodging amenity", lang = "en")]
        comment = locstr("""In property and land use planning, amenity (lat. amoenitās “pleasantness, delightfulness”) is something considered to benefit a location, contribute to its enjoyment, and thereby increase its value.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Amenity) """, lang = "en")
        equivalent_to = [acco.AccommodationFeature]
        seeAlso = [create_external_resource("Amenity", dbr)]
    
    class Bathroom(tao_solo.LocationAmenity):
        label = [locstr("Bathroom", lang = "en")] + ho.Bathroom.label ## Enrich Bathroom with multilang labels from Hospitality 
        altLabel = [
            locstr("Bath", lang = "en")]
        comment = locstr("""A bathroom or washroom is a room, typically in a home or other residential building, that contains either a bathtub or a shower (or both).
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Bathroom) """, lang = "en")
        seeAlso = [create_external_resource("Bathroom", dbr)]
    
    class Service(tao_solo.LocationAmenity):
        label = locstr("Service", lang = "en")
        comment = locstr("""A service is a transaction in which no physical goods are transferred from the seller to the buyer.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Service_(economics)) """, lang = "en")
        seeAlso = [create_external_resource("Service_(economics)", dbr)]
    
    class Accessibility(tao_solo.LocationAmenity):
        label = locstr("Accessibility", lang = "en")

    class SafetyAndSecurity(tao_solo.LocationAmenity):
        label = locstr("Safety and security", lang = "en")
        
    class BathroomAmenity(tao_solo.LocationAmenity):
        label = locstr("Bathroom amenity", lang = "en")
    
    class AmenityArea(tao_solo.LocationAmenity, schema.Place):
        label = label = locstr("Amenity area", lang = "en")
        comment = locstr("""AMENITY AREA means an area or areas intended for use for recreation or aesthetic purposes within the boundaries of a lot and may include landscaped open space areas, patios, balconies, communal lounges, swimming pools, recreation facilities, and any other area which may be used for recreational or aesthetic purposes, but does not include any driveways or parking area.
        (Source Law Insider, see https://www.lawinsider.com/dictionary/amenity-area""")
    
    class ChangingRoom(tao_solo.AmenityArea):
        label = label = locstr("Changing room", lang = "en")  
        altLabel = [
            locstr("Dressing room", lang = "en"),
            locstr("Locker room", lang = "en")]
        comment = locstr("""A changing room, locker room, dressing room (usually in a sports, theater or staff context) or changeroom (regional use) is a room or area designated for changing one's clothes. Changing rooms are provided in a semi-public situation to enable people to change clothes with varying degrees of privacy.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Changing_room)""")
        seeAlso = [ create_external_resource("Changing_room", dbr)]
        
    class Equipment(tao_solo.LocationAmenity):
        label = label = locstr("Equipment", lang = "en")
        comment = locstr("""Equipment most commonly refers to a set of tools or other objects commonly used to achieve a particular objective. Different jobs require different kinds of equipment.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Equipment""")
        seeAlso = [ create_external_resource("Equipment", dbr)]
    
            
    class CookingEquipment(tao_solo.Equipment):
        label = locstr("Cooking equipment", lang = "en")
        
    class FoodAndDrinks(tao_solo.LocationAmenity):
        label = locstr("Food and drinks", lang = "en")        
        
    class HomeAppliance(tao_solo.Equipment):
        label = locstr("Home appliance", lang = "en")
        comment = locstr("""A home appliance, also referred to as a domestic appliance, an electric appliance or a household appliance,[1] is a machine which assists in household functions[2] such as cooking, cleaning and food preservation.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Equipment)""")
        seeAlso = [ create_external_resource("Home_appliance", dbr)]
    
    class ChildCare(tao_solo.LocationAmenity):
        label = label = locstr("Child care", lang = "en")
        comment = locstr("""Child care, otherwise known as day care, is the care and supervision of a child or multiple children at a time, whose ages range from two weeks to eighteen years. 
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Child_care)""")
        seeAlso = [ create_external_resource("Child_care", dbr)]

    class Furniture(tao_solo.LocationAmenity):
        label = label = locstr("Furniture", lang = "en")
        comment = locstr("""Furniture refers to movable objects intended to support various human activities such as seating (e.g., Stools, Chairs, and sofas), eating (tables), storing items, eating and/or working with an item, and sleeping (e.g., beds and hammocks). 
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Furniture)""")
        seeAlso = [ create_external_resource("Furniture", dbr)]        

        
    class GameEquipment(tao_solo.Equipment):
        label = locstr("Game", lang = "en")
        altLabel = locstr("Game equipment", lang = "en")
        comment = locstr("""A game is a structured form of play, usually undertaken for entertainment or fun, and sometimes used as an educational tool.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Game""")
        seeAlso = [ create_external_resource("Game", dbr)]
    
    class CommonArea(tao_solo.AmenityArea, schema.Place):
        label = label = locstr("Common area", lang = "en")
        comment = locstr("""A common area is, in real estate or real property law, the "area which is available for use by more than one person.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Common_area)""")
        seeAlso = [ create_external_resource("Common_area", dbr)]
        
    
    #class CommonAmenity(tao_solo.LocationAmenity):
    #    label = locstr("Common feature", lang = "en")
    #    altLabel = locstr("Common amenity", lang = "en")
    #    comment = locstr("""Common Amenities/features means those facilities, common areas, and services in the Building and/or Property available for use by, or supplied to the tenants on a non-exclusive basis and include interalia lifts, air-conditioning, security services, common areas, visitor’s parking and such like.
    #    (Source Law Insider, see https://www.lawinsider.com/dictionary/common-amenities) """, lang = "en")
        
    class HotTub(tao_solo.CommonArea):
        label = [locstr("Hot tub", lang = "en")] + ho.HydromassageJacuzzi.label + ho.ExteriorJacuzzi.label ## Enrich HotTub with multilang labels from Hospitality 
        altLabel = [locstr("Jacuzzi", lang = "en"), locstr("Hydromassage Jacuzzi", lang = "en")]
        comment = locstr("""A hot tub is a large tub full of water used for hydrotherapy, relaxation or pleasure. Some have powerful jets for massage purposes. Hot tubs are sometimes also known as "spas" or by the trade name Jacuzzi.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Hot_tub) """, lang = "en")
        seeAlso = [ create_external_resource("Hot_tub", dbr)]      
   
    class ShoeShiner(tao_solo.Service):
        label = [locstr("ShoeShiner", lang = "en")] + ho.ShoeShine.label ## Enrich ShoeShiner with multilang labels from Hospitality 
        altLabel = [locstr("ShoeShine", lang = "en")]
        comment = locstr("""Shoeshiner or boot polisher is an occupation in which a person cleans and buffs shoes and then applies a waxy paste to give a shiny appearance and a protective coating.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/ShoeShiner) """, lang = "en")
        seeAlso = [ create_external_resource("ShoeShiner", dbr)]

    class HeatingSystem(tao_solo.HomeAppliance):
        label = [locstr("Heating System", lang = "en")] + ho.HeatingSystem.label ## Enrich HeatingSystem with multilang labels from Hospitality 
        comment = locstr("""A heating system is a mechanism for maintaining temperatures at an acceptable level; by using thermal energy within a home, office, or other dwelling.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Heating_system) """, lang = "en")
        seeAlso = [ create_external_resource("Heating_system", dbr)]
        
        
    class PublicToilet(tao_solo.CommonArea):
        label = locstr("Public toilet", lang = "en")
        comment = locstr("""A public toilet is a room or small building with toilets (or urinals) and sinks for use by the general public. 
        The facilities are available to customers, travelers, employees of a business, school pupils and prisoners and are commonly separated into male and female toilets, although some are unisex, especially for small or single-occupancy public toilets.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Public_toilet)""")
        seeAlso = [ create_external_resource("Public_toilet", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Toilet")       
    
    class GolfCourse(tao_solo.CommonArea):
        altLabel = locstr("Golf courses", lang = "en") ## main label already imported from Schema.org
        comment = locstr("""A golf course is the grounds where the sport of golf is played.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Golf_course)""")
        seeAlso = [ create_external_resource("Golf_course", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Golf")

    class Gym(tao_solo.CommonArea):
        label = [locstr("Gym", lang = "en")] +  ho.Gym.label ## Enrich Gym imported with multilang labels from Hospitality
        comment = locstr("""A gymnasium, also known as a gym, is a covered location for athletics. The word is derived from the ancient Greek gymnasium.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Gym)""")
        seeAlso = [ create_external_resource("Gym", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Gym")
    
    class DaySpa(tao_solo.CommonArea):
        label = (tao_solo.DaySpa.label + ho.SpaCentre.label)[:-1] ## Enrich DaySpa imported from Schema.org add multilang labels from Hospitality but remove wrong label "WellnessCentre"
        altLabel = locstr("Spa center", lang = "en")
        comment = locstr("""A day spa is a business that provides a variety of services for the purpose of improving health, beauty and relaxation through personal care treatments such as hair, massages and facials. 
        A day spa is different from a beauty salon in that it contains facilities such as a sauna, pool, steam room, or whirlpool that guests may use in addition to their treatment.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Day_spa)""")
        seeAlso = [ create_external_resource("Day_spa", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Spa")
    
    class TurkishBath(tao_solo.CommonArea):
        label = [locstr("Turkish bath", lang = "en")] +  ho.TurkishBath.label ## Enrich TurkishBath imported with multilang labels from Hospitality
        altLabel = locstr("Hammam", lang = "en")
        comment = locstr("""A hammam (Arabic: حمّام, romanized: ḥammām, Turkish: hamam) is a type of steam bath or a place of public bathing associated with the Islamic world.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Sauna)""")
        seeAlso = [ create_external_resource("Hammam", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Turkish bath")
    
    class Sauna(tao_solo.CommonArea):
        label = [locstr("Sauna", lang = "en")] +  ho.SpaCentre.label ## Enrich Sauna imported with multilang labels from Hospitality
        comment = locstr("""A sauna (/ˈsɔːnə, ˈsaʊnə/, Finnish: [ˈsɑu̯nɑ]), or sudatory, is a small room or building designed as a place to experience dry or wet heat sessions, or an establishment with one or more of these facilities. 
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Sauna)""")
        seeAlso = [ create_external_resource("Sauna", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Sauna")
    
    class ParkingFacility(tao_solo.CommonArea):
        label = (tao_solo.ParkingFacility.label + ho.Parking.label) ## Enrich ParkingFacility imported from Schema.org add multilang labels from Hospitality 
        altLabel = locstr("Parking", lang = "en")
        equivalent_to = tao_builder.acco_feature_equivalent_class("Parking")        
    
    class Restaurant(tao_solo.CommonArea):
        label = (tao_solo.Restaurant.label + ho.Restaurant.label)[:-1] ## Enrich Restaurant imported from Schema.org add multilang labels from Hospitality but remove wrong label "WellnessCentre"
        comment = locstr("""A restaurant is a business that prepares and serves food and drinks to customers.
        (Source Wikipedia, see https://en.wikipedia.org/wiki/Restaurant)""")
        seeAlso = [ create_external_resource("Restaurant", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Restaurant")        

    class SwimmingPool(tao_solo.AmenityArea):
        label = [locstr("Swimming pool", lang = "en")]
        comment = """A swimming pool, swimming bath, wading pool, paddling pool, or simply pool is a structure designed to hold water to enable swimming or other leisure activities 
        (Source Wikipedia: https://en.wikipedia.org/wiki/Swimming_pool"""
        altLabel = locstr("pool", lang = "en")
        seeAlso = [ create_external_resource("Swimming_pool", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Swimming pool")
        
    class FitnessService(tao_solo.Service):
        label = locstr("Fitness service", lang = "en")
        comment = locstr("""Fitness service means any service treatment, advice, diagnosis or instruction relating to physical fitness, including but not limited to diet, cardio-vascular fitness, body building or physical training programs and which you perform as or on behalf of the named insured.
        (Source Law Insider, see https://www.lawinsider.com/dictionary/fitness-services)""")
               
    class KidsService(tao_solo.Service):
        label = locstr("Kids service", lang = "en")
        altLabel = locstr("Service for kids", lang = "en")
        comment = locstr("""Kids service referes to any service relating to kids care and activities.""")
    
    class HairCare(tao_solo.Service):
        label = locstr("Hair care", lang = "en")
        comment = locstr("""Hair care is an overall term for hygiene and cosmetology involving the hair.
        (Source Wikipedia: https://en.wikipedia.org/wiki/Hair_care)""")
        seeAlso = [ create_external_resource("Hair_care", dbr)]
        equivalent_to = tao_builder.acco_feature_equivalent_class("Hair care")
    
    class AmusementService(tao_solo.Service):
        label = locstr("Amusement service", lang = "en")      
        comment = locstr("""Amusement services means the provision of amusement, entertainment, or recreation,.
        (Source Law Insider, see https://www.lawinsider.com/dictionary/amusement-services)""")
 
    class LaundryService(tao_solo.Service):
        label = [locstr("Laundry Service", lang = "en")] +  ho.LaundryService.label ## Enrich LaundryService imported with multilang labels from Hospitality

    class SpokenLanguage(tao_solo.Service):
        label = [locstr("Spoken language", lang = "en")]

In [24]:
len(list(tao_ontology.classes()))

254

### Import subbclasses of ho:RoomFacility as tao:LodgingFacility subclasses
We now import from Hontology some classes useful to model accommodations.

In [25]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ ho.Bed ]
only_include_classes = [ho.Kettle]
hontology_room_facility_classes = {}
duplicate_class_hyerarchy(ho.RoomFacility, hontology_room_facility_classes, tao_solo.CookingEquipment, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, include = only_include_classes ,exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

In [26]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ ho.Bed ]
only_include_classes = [ho.Kitchen, ho.Terrace]
hontology_room_facility_classes = {}
duplicate_class_hyerarchy(ho.RoomFacility, hontology_room_facility_classes, tao_solo.AmenityArea, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, include = only_include_classes ,exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

In [27]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ ho.Bed ]
only_include_classes = [ho.AirConditioning, ho.Television, ho.Refrigerator, ho.CoffeeMachine, 
                        ho.Telephone, ho.Iron, ho.EletricKettle, ho.Lamp, ho.Phone, ho.Refrigerator]
hontology_room_facility_classes = {}
duplicate_class_hyerarchy(ho.RoomFacility, hontology_room_facility_classes, tao_solo.HomeAppliance, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, include = only_include_classes ,exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

### Import subbclasses of ho:BathRoomFacility as tao:BathroomAmenity subclasses
We now import from Hontology some classes useful to model amenities.

In [28]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
hontology_bathroom_facility_classes = {}
duplicate_class_hyerarchy(ho.BathRoomFacility, hontology_room_facility_classes, tao_solo.BathroomAmenity, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

### Import subbclasses of ho:InternalFacility as tao:CommonArea subclasses
We now import from Hontology some classes useful to model amenities.

In [29]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ho.Bathroom, ho.Cot, ho.Elevator, ho.Hairdresser, ho.HeatingSystem, ho.IndoorSwimmingPool, ho.InternetAccess,
                   ho.KidsClub, ho.LaundryService, ho.LaundryRoom, ho.Meeting, ho.Parking, ho.Restaurant, ho.ShoeShine,
                   ho.SpaCentre]
hontology_internal_facility_to_common_area_classes = {}
duplicate_class_hyerarchy(ho.InternalFacility, hontology_internal_facility_to_common_area_classes, tao_solo.CommonArea, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

### Import subbclasses of ho:ExternalFacility as tao:CommonArea subclasses
We now import from Hontology some classes useful to model amenities.

In [30]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ho.ExteriorJacuzzi, ho.Golf, ho.InternetAccess, ho.OutdoorSwimmingPool, ho.Parking, ho.Square]
hontology_internal_facility_to_common_area_classes = {}
duplicate_class_hyerarchy(ho.ExternalFacility, hontology_internal_facility_to_common_area_classes, tao_solo.CommonArea, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

### Import subbclasses of ho:Service as tao:Service subclasses
We now import from Hontology some classes useful to model amenities.

In [31]:
disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
    ],
    "do_not_disjoin": [ #remove from disjoint Axioms in special cases (use tuples to define which classes should neve be disjoint)
    ]
}
exclude_classes = [ho.HoneymoonSuite, ho["Guest-chosenDailyNewspaperDelivery"]]
hontology_service_classes = {}
duplicate_class_hyerarchy(ho.Service, hontology_service_classes, tao_solo.Service, 
                          marshallers = [annotation_copier, labeller_by_class_name, provenance_annotator], 
                          exclude = exclude_classes, exclude_disjoint = disjoin_config, 
                          namespace = tao_solo, class_wrapper=tao_builder.acco_feature_class, disjoint=False)

## Save all classes created so far as a csv file

In [32]:
classes_df = generate_class_hierarchy(tao_solo)

In [33]:
classes_df.to_csv("class_mapping/location_amenities_base_classes.csv", index = False)

## Enrich with text analysis

#### AirBnB listings description most frequent words

In [34]:
df = pd.read_csv("./enrichment/airbnb_listing_sardinia_tf_ng_2_2_mm_8_mn_005_pos_20210414 - Most frequent entites.csv")

In [35]:
df = df[df['selected'] == 'yes'].fillna("")

In [36]:
dummy = df.apply(lambda r: tao_builder.process_entity(tao_solo, r['entity'],r['parent_class'],r['class'], r['type'], r['is_amenity'], provenance = "AirBnB listings word frequency analysis", comment_text = "Enriched with AirBnB listings word frequency analysis"), axis=1)

#### AirBnB listings description most frequent digrams

In [37]:
df = pd.read_csv("./enrichment/airbnb_listing_sardinia_tf_ng_2_2_mm_8_mn_005_pos_20210414 - digram entities.csv")

In [38]:
df = df[df['selected'] == 'yes'].fillna("")

In [39]:
dummy = df.apply(lambda r: tao_builder.process_entity([tao_solo, acco], r['entity'],r['parent_class'],r['class'], r['type'], r['is_amenity']
                                 ,provenance = "AirBnB listings digram frequency analysis", comment_text = "Enriched with AirBnB listings word frequency analysis"), axis=1)

#### Booking.com features lists

In [40]:
df = pd.read_csv("./enrichment/booking_facilities_and_airbnb_amenities - booking_facilities.csv")

In [41]:
df = df[df['selected'] == 'yes'].fillna("")

In [42]:
dummy = df.apply(lambda r: tao_builder.process_entity([tao_solo, acco], r['entity'],r['parent_class'],r['class'], r['type'], r['is_amenity']
                                 ,provenance = "Booking.com features lists extraction.", comment_text = "Enriched Booking.com features lists extraction"), axis=1)

In [43]:
df = pd.read_csv("./enrichment/booking_facilities_and_airbnb_amenities - airbnb_amenities.csv")

In [44]:
df = df[df['selected'] == 'yes'].fillna("")

In [45]:
dummy = df.apply(lambda r: tao_builder.process_entity(tao_solo, r['entity'],r['parent_class'],r['class'], r['type'], r['is_amenity']
                                 ,provenance = "AirBnB amenities lists extraction.", comment_text = "Enriched AirBnB amenities lists extraction"), axis=1)

### Apply disjoint to Location Amenity

In [46]:
exclude_from_exploration = [
    tao_solo.HygieneProducts, ## BathroomAmenity
    tao_solo.Toilet, ## BathroomAmenity
    tao_solo.Television, ## Equipment
    tao_solo.ClothingStorage, ## Furniture
    tao_solo.Safe, ## SafetyAndSecurity
    tao_solo.View, ## AmenityArea
    tao_solo.Kitchen, ## AmenityArea
    tao_solo.CommonArea, ## AmenityArea
    tao_solo.CookingEquipment, ## Equipment
    tao_solo.BeautyService, ## Service
    tao_solo.DrycleaningService, ## Service
    tao_solo.HairCare ## Service
]

disjoin_config = {
    "*": [ #remove from disjoint Axioms in any case
        tao_solo.CommonArea,
        tao_solo.View
    ],
    "do_not_disjoin": [
        (tao_solo.VendingMachineDrinks,tao_solo.VendingMachineSnacks), ## AmenityArea
        (tao_solo.Veranda,tao_solo.Terrace), ## AmenityArea
        (tao_solo.LivingArea,tao_solo.DiningArea, tao_solo.OutdoorDiningArea, tao_solo.MeetingBanquetFacilities), ## AmenityArea
        (tao_solo.FreeToiletries,tao_solo.HygieneProducts), ## BathroomAmenity
        (tao_solo.Sink,tao_solo.Tap), ## BathroomAmenity
        (tao_solo.LiveMusic,tao_solo.LivePerformance), ## Service
        (tao_solo.FitnessClasses,tao_solo.Aerobics, tao_solo.FullBodyMassage, tao_solo.PersonalTrainer), ## Service
        (tao_solo.KeyAccess,tao_solo.KeyCardAccess, tao_solo.Keypad, tao_solo.SmartLock), ## SafetyAndSecurity
        (tao_solo.DrycleaningService,tao_solo.LaundryService), ## Service
    ]
}
disjoin_class_hyerarchy(tao_solo.ChildCare, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.AmenityArea, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.BathroomAmenity, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.ChildCare, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.Equipment, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.FoodAndDrinks, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.Furniture, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.SafetyAndSecurity, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
disjoin_class_hyerarchy(tao_solo.Service, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)
#disjoin_class_hyerarchy(tao_solo.Accessibility, namespace = tao_solo, exclude_disjoint = disjoin_config, exclude = exclude_from_exploration)


## Save all classes created so far as a csv file

In [47]:
classes_df = generate_class_hierarchy(tao_solo)

In [48]:
classes_df.to_csv("class_mapping/location_amenities_enriched_classes.csv", index = False)

## Save ontology file output

In [49]:
tao_ontology.save(file = "output_ontology/tao.rdf", format = "rdfxml")